
- 월단위 데이터 테이블 (모델 학습용)
- 일일단위 테이블 (모델 학습용)

station_id | use_mm | hour |  get_on | get_off | total
|:--|:--|:--|:--|:--|:--
1001 | 202111 | 9 | 48518 | 131911 | 180429
1001 | 202111 | 8 | 61857 | 196998 | 258855
1001 | 202112 | 8 | 65000 | 205000 | 270000
1001 | 202112 | 9 | 50000 | 135000 | 185000
1002 | 202111 | 8 | 30000 | 150000 | 180000
1002 | 202111 | 9 | 32000 | 140000 | 172000

- 일/시간단위 예측 결과 (서비스용)
```SQL
CREATE TABLE subway_congestion_pred (
  station_id    INT         NOT NULL,
  service_date  DATE        NOT NULL,
  hour          TINYINT     NOT NULL,      -- 0~23
  predicted_total INT       NOT NULL,
  updated_at    TIMESTAMP   NOT NULL
                   DEFAULT CURRENT_TIMESTAMP
                   ON UPDATE CURRENT_TIMESTAMP,
  PRIMARY KEY (station_id, service_date, hour)
);
```

In [42]:
import psycopg2
import pandas as pd

conn_params = {
    "host": "localhost",
    "port": 5432,
    "dbname": "seoulmoa",
    "user": "airflow",
    "password": "airflow"
}

try:
    # connect
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor()
    print("▶︎ PostgreSQL에 연결되었습니다.")

    # search db, schema
    cur.execute("SELECT current_database(), current_schema();")
    db, schema = cur.fetchone()
    print(f"연결된 DB: {db}, search_path 스키마: {schema}")

    # schema list
    cur.execute("SELECT schema_name FROM information_schema.schemata;")
    schemata = [row[0] for row in cur.fetchall()]
    print("▶스키마:", schemata)


    # st search_path
    cur.execute("SET search_path TO datawarehouse;")
    print("search_path --> datawarehouse 설정.")

    # search table
    cur.execute("""
      SELECT table_name
      FROM information_schema.tables
      WHERE table_schema = 'datawarehouse';
    """)
    tables = [row[0] for row in cur.fetchall()]
    print("▶︎ datawarehouse 스키마의 테이블들:", tables)

    # act query

    df = pd.read_sql_query(f'SELECT * FROM "dailySubwaystation"', conn)
except psycopg2.Error as e:
    print(e)
    raise
finally:
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()

▶︎ PostgreSQL에 연결되었습니다.
연결된 DB: seoulmoa, search_path 스키마: public
▶스키마: ['pg_toast', 'pg_catalog', 'public', 'information_schema', 'datawarehouse']
search_path --> datawarehouse 설정.
▶︎ datawarehouse 스키마의 테이블들: ['Event', 'MontlySubwaystation', 'SubwayStation', 'dailySubwaystation']


C:\Users\brian\AppData\Local\Temp\ipykernel_30464\827382572.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'SELECT * FROM "dailySubwaystation"', conn)


In [ ]:
import requests
import pandas as pd
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
api = ''
result =[]
for page in range(1,622):
    url = f'http://openapi.seoul.go.kr:8088/{api}/json/CardSubwayTime/{page}/{page}/202503/'
    res = requests.get(url =url)
    data = res.json()
    data = data['CardSubwayTime']['row']
    result.extend(data)
print(result)

result = pd.DataFrame(result)
feature_columns = ['station_id', 'use_month', 'hour', 'sbwy_rout_ln_nm', 'get_on', 'get_off']

# 시간대 컬럼 리스트 추출
hourly_on_off_columns = [col for col in result.columns if col.startswith('HR_')]

# melt
feature_table_melted = pd.melt(
    result,
    id_vars=['USE_MM', 'STTN', 'SBWY_ROUT_LN_NM'],
    value_vars=hourly_on_off_columns,
    var_name='hour_type',
    value_name='count'
)

# 안전한 split
split_columns = feature_table_melted['hour_type'].str.split('_', expand=True)
feature_table_melted['hour'] = split_columns[1].astype(int)
feature_table_melted['ride_type'] = split_columns[3]
feature_table_melted.drop(columns='hour_type',inplace=True)

feature_table_melted['get_on'] = feature_table_melted.apply(
    lambda row: row['count'] if row['ride_type'] == 'ON' else 0, axis=1
)

feature_table_melted['get_off'] = feature_table_melted.apply(
    lambda row: row['count'] if row['ride_type'] == 'OFF' else 0, axis=1
)

# 3. 이제 hour 단위로 ON/OFF 한 줄로 묶기 (groupby)
feature_table_grouped = feature_table_melted.groupby(
    ['USE_MM', 'STTN', 'SBWY_ROUT_LN_NM', 'hour'],
    as_index=False
)[['get_on', 'get_off']].sum()

# 4. 필요한 컬럼 정리 (rename 등)
feature_table_grouped.rename(columns={
    'USE_MM': 'use_month',
    'STTN': 'station_id',
    'SBWY_ROUT_LN_NM': 'sbwy_rout_ln_nm'
}, inplace=True)

# 5. total 컬럼 추가
feature_table_grouped['total'] = feature_table_grouped['get_on'] + feature_table_grouped['get_off']

[{'USE_MM': '202503', 'SBWY_ROUT_LN_NM': '6호선', 'STTN': '연신내', 'HR_4_GET_ON_NOPE': 0.0, 'HR_4_GET_OFF_NOPE': 0.0, 'HR_5_GET_ON_NOPE': 0.0, 'HR_5_GET_OFF_NOPE': 0.0, 'HR_6_GET_ON_NOPE': 0.0, 'HR_6_GET_OFF_NOPE': 0.0, 'HR_7_GET_ON_NOPE': 2.0, 'HR_7_GET_OFF_NOPE': 0.0, 'HR_8_GET_ON_NOPE': 6.0, 'HR_8_GET_OFF_NOPE': 0.0, 'HR_9_GET_ON_NOPE': 5.0, 'HR_9_GET_OFF_NOPE': 0.0, 'HR_10_GET_ON_NOPE': 8.0, 'HR_10_GET_OFF_NOPE': 0.0, 'HR_11_GET_ON_NOPE': 2.0, 'HR_11_GET_OFF_NOPE': 0.0, 'HR_12_GET_ON_NOPE': 4.0, 'HR_12_GET_OFF_NOPE': 0.0, 'HR_13_GET_ON_NOPE': 5.0, 'HR_13_GET_OFF_NOPE': 0.0, 'HR_14_GET_ON_NOPE': 3.0, 'HR_14_GET_OFF_NOPE': 0.0, 'HR_15_GET_ON_NOPE': 1.0, 'HR_15_GET_OFF_NOPE': 0.0, 'HR_16_GET_ON_NOPE': 7.0, 'HR_16_GET_OFF_NOPE': 0.0, 'HR_17_GET_ON_NOPE': 8.0, 'HR_17_GET_OFF_NOPE': 0.0, 'HR_18_GET_ON_NOPE': 2.0, 'HR_18_GET_OFF_NOPE': 0.0, 'HR_19_GET_ON_NOPE': 2.0, 'HR_19_GET_OFF_NOPE': 0.0, 'HR_20_GET_ON_NOPE': 3.0, 'HR_20_GET_OFF_NOPE': 0.0, 'HR_21_GET_ON_NOPE': 0.0, 'HR_21_GET_OFF_NOPE': 

In [18]:
df = feature_table_grouped
# 1. 시간대-역별 total 합계
hour_station_total = df.groupby(['hour', 'station_id'])['total'].sum().reset_index()

# 2. 시간대별 전체 total
hour_total_sum = hour_station_total.groupby('hour')['total'].sum().reset_index(name='hour_total_sum')

# 3. 병합해서 비율 계산
hourly_ratio_df = hour_station_total.merge(hour_total_sum, on='hour')
hourly_ratio_df['hour_ratio'] = hourly_ratio_df['total'] / hourly_ratio_df['hour_total_sum']

# 4. 필요한 컬럼만 추리기
hourly_ratio_df = hourly_ratio_df[['hour', 'station_id', 'hour_ratio']]
hourly_ratio_df = hourly_ratio_df.rename(columns={
    'station_id':'name'
})

In [ ]:
df

,service_date,line,name,get_on_d,get_off_d,REG_YMD
0,20250417,1호선,서울역,76692,71176,20250420
1,20250417,1호선,시청,30571,30787,20250420
2,20250417,1호선,종각,44684,44744,20250420
3,20250417,1호선,종로3가,29455,26148,20250420
4,20250417,1호선,종로5가,25548,25548,20250420
...,...,...,...,...,...,...
612,20250417,중앙선,오빈,352,357,20250420
613,20250417,중앙선,양평,3453,3503,20250420
614,20250417,중앙선,원덕,433,441,20250420
615,20250417,중앙선,용문,2199,2166,20250420


In [53]:
cond = hourly_ratio_df['name'] == '불암산'
hourly_ratio_df.loc[cond]

,hour,name,hour_ratio


In [55]:
# 전체 평균 비율로 fallback 채우기
mean_ratio = hourly_ratio_df.groupby('hour')['hour_ratio'].mean().reset_index()

hourly_synth = (
    df
    .merge(hourly_ratio_df, on='name', how='left')
    .merge(mean_ratio, on='hour', suffixes=('', '_mean'))
    .assign(
        final_ratio=lambda d: d['hour_ratio'].fillna(d['hour_ratio_mean']),
        synth_total=lambda d: d['get_on_d'] * d['final_ratio']
    )
    .drop(columns=['hour_ratio', 'hour_ratio_mean', 'final_ratio'])
)


In [60]:
display(hourly_synth)

,service_date,line,name,get_on_d,get_off_d,REG_YMD,hour,synth_total
0,20250417,1호선,서울역,76692,71176,20250420,0.0,866.933764
1,20250417,1호선,서울역,76692,71176,20250420,1.0,823.301447
2,20250417,1호선,서울역,76692,71176,20250420,2.0,0.000000
3,20250417,1호선,서울역,76692,71176,20250420,3.0,0.000000
4,20250417,1호선,서울역,76692,71176,20250420,4.0,708.052875
...,...,...,...,...,...,...,...,...
14731,20250417,중앙선,지평,78,84,20250420,19.0,0.001123
14732,20250417,중앙선,지평,78,84,20250420,20.0,0.000012
14733,20250417,중앙선,지평,78,84,20250420,21.0,0.000654
14734,20250417,중앙선,지평,78,84,20250420,22.0,0.000021


In [64]:
display(result)

,USE_MM,SBWY_ROUT_LN_NM,STTN,HR_4_GET_ON_NOPE,HR_4_GET_OFF_NOPE,HR_5_GET_ON_NOPE,HR_5_GET_OFF_NOPE,HR_6_GET_ON_NOPE,HR_6_GET_OFF_NOPE,HR_7_GET_ON_NOPE,...,HR_23_GET_OFF_NOPE,HR_0_GET_ON_NOPE,HR_0_GET_OFF_NOPE,HR_1_GET_ON_NOPE,HR_1_GET_OFF_NOPE,HR_2_GET_ON_NOPE,HR_2_GET_OFF_NOPE,HR_3_GET_ON_NOPE,HR_3_GET_OFF_NOPE,JOB_YMD
0,202503,6호선,연신내,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20250403
1,202503,1호선,서울역,672.0,53.0,10025.0,9052.0,26032.0,57132.0,86883.0,...,28364.0,3694.0,8024.0,7.0,178.0,0.0,0.0,0.0,0.0,20250403
2,202503,1호선,시청,86.0,1.0,1997.0,5455.0,3779.0,25804.0,7720.0,...,3996.0,498.0,1158.0,1.0,35.0,0.0,1.0,0.0,0.0,20250403
3,202503,1호선,종각,141.0,2.0,4180.0,6021.0,4437.0,34064.0,7104.0,...,4664.0,1796.0,1353.0,0.0,0.0,0.0,0.0,0.0,0.0,20250403
4,202503,1호선,종로3가,207.0,20.0,3687.0,2495.0,4045.0,11669.0,5769.0,...,5468.0,1550.0,2286.0,1.0,43.0,0.0,0.0,0.0,0.0,20250403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,202503,중앙선,오빈,1.0,0.0,277.0,25.0,453.0,324.0,1133.0,...,161.0,1.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,20250403
617,202503,중앙선,양평,115.0,3.0,2145.0,43.0,4375.0,1078.0,11035.0,...,3733.0,73.0,1192.0,0.0,0.0,0.0,0.0,0.0,0.0,20250403
618,202503,중앙선,원덕,67.0,1.0,272.0,13.0,513.0,107.0,919.0,...,272.0,2.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,20250403
619,202503,중앙선,용문,415.0,0.0,1124.0,9.0,2472.0,385.0,4333.0,...,1449.0,0.0,701.0,0.0,0.0,0.0,0.0,0.0,0.0,20250403


In [ ]:
x_train = pd.concat([])

,service_date,line,name,get_on_d,get_off_d,REG_YMD,hour,synth_total
0,20250417,1호선,서울역,76692,71176,20250420,0.0,866.933764
1,20250417,1호선,서울역,76692,71176,20250420,1.0,823.301447
2,20250417,1호선,서울역,76692,71176,20250420,2.0,0.000000
3,20250417,1호선,서울역,76692,71176,20250420,3.0,0.000000
4,20250417,1호선,서울역,76692,71176,20250420,4.0,708.052875
...,...,...,...,...,...,...,...,...
14731,20250417,중앙선,지평,78,84,20250420,19.0,0.001123
14732,20250417,중앙선,지평,78,84,20250420,20.0,0.000012
14733,20250417,중앙선,지평,78,84,20250420,21.0,0.000654
14734,20250417,중앙선,지평,78,84,20250420,22.0,0.000021


In [ ]:
encoder = LabelEncoder()
scaler = MinMaxScaler()
one_hot_col = ['use_month','station_id','sbwy_rout_ln_nm']
min_max_col = ['get_on','get_off']
for v in one_hot_col:
    feature_table_grouped[v] = encoder.fit_transform(feature_table_grouped[v])
for w in min_max_col:
    feature_table_grouped[w] = scaler.fit_transform(feature_table_grouped[[w]])
display(feature_table_grouped)

,use_month,station_id,sbwy_rout_ln_nm,hour,get_on,get_off,total
0,0,0,0,0,0.014297,0.030089,11718.0
1,0,0,0,1,0.000027,0.000667,185.0
2,0,0,0,2,0.000000,0.000000,0.0
3,0,0,0,3,0.000000,0.000000,0.0
4,0,0,0,4,0.002601,0.000199,725.0
...,...,...,...,...,...,...,...
91,0,3,0,19,0.447665,0.104157,143441.0
92,0,3,0,20,0.427617,0.056852,125646.0
93,0,3,0,21,0.382465,0.039685,109402.0
94,0,3,0,22,0.247506,0.027918,71394.0


In [ ]:
train_x = feature_table_grouped.drop(columns=['get_on','get_off','total'])
train_y = feature_table_grouped[['total']]
print(train_y)

       total
0    11718.0
1      185.0
2        0.0
3        0.0
4      725.0
..       ...
91  143441.0
92  125646.0
93  109402.0
94   71394.0
95   31089.0

[96 rows x 1 columns]


In [ ]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(train_x,train_y,test_size=0.3,random_state=2024)
model = RandomForestRegressor()
model.fit(X_TRAIN,Y_TRAIN)
y_pred = model.predict(X_TEST)
print(r2_score(y_pred,Y_TEST))
print(Y_TEST)

c:\Users\brian\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8847102225691039
       total
21  188874.0
35   75878.0
80  277400.0
25      36.0
16  278168.0
14  212140.0
24    1656.0
51       0.0
92  125646.0
77   10201.0
6    83164.0
34   66882.0
5    19077.0
17  371411.0
67       2.0
15  258136.0
88  146971.0
58       8.0
19  264683.0
3        0.0
84   96901.0
9   285125.0
64       7.0
23   65245.0
49       0.0
2        0.0
89  221885.0
59       2.0
72    3149.0


In [ ]:
user_input = {
    'use_month':0, # dummy
    'station_id':1,
    'sbwy_rout_ln_nm': 4,
    'hour':19
}

model_input = np.array([[0,1,4,19]])
print(model.predict(model_input))

[1658.14]


c:\Users\brian\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
# get_subway_daily_data.get_data
import requests
import pandas as pd
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
api = '58536c526362726936366c51544b62'
result =[]
for page in range(1,2):
    url = f'http://openapi.seoul.go.kr:8088/{api}/json/CardSubwayStatsNew/1/1/20250417/'
    res = requests.get(url =url)
    data = res.json()
    #data = data['CardSubwayStatsNew']['row']
    #result.extend(data)
pprint(data)

{'CardSubwayStatsNew': {'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
                        'list_total_count': 617,
                        'row': [{'GTOFF_TNOPE': 71176.0,
                                 'GTON_TNOPE': 76692.0,
                                 'REG_YMD': '20250420',
                                 'SBWY_ROUT_LN_NM': '1호선',
                                 'SBWY_STNS_NM': '서울역',
                                 'USE_YMD': '20250417'}]}}


In [ ]:
df = pd.DataFrame(data['CardSubwayStatsNew']['row'])
df

,USE_YMD,SBWY_ROUT_LN_NM,SBWY_STNS_NM,GTON_TNOPE,GTOFF_TNOPE,REG_YMD
0,20250417,1호선,서울역,76692.0,71176.0,20250420
